In [3]:
# ! pip install censusdata
# ! pip install census
# ! pip install us

In [2]:
import pandas as pd
# import censusdata
import glob
import os
from tqdm import tqdm
# import matplotlib.pyplot as plt
import random
import geopandas as gpd
from datetime import datetime


# from matplotlib.font_manager import FontManager
# fm = FontManager()
# mat_fonts = set(f.name for f in fm.ttflist)
# print(mat_fonts)

import matplotlib.pyplot as plt

# plt.rcParams['font.sans-serif'] = ['DengXian']
plt.rcParams.update({'font.size': 16})
pd.set_option('display.max_columns', None)  # show all columns
pd.set_option("display.max_colwidth", 100)

# Load demographic data

In [10]:
# # weekly residing files
# weekly_residing_all_df = pd.read_csv(r'J:\weekly_patterns_20211211\Panel_overview_data\WeeklyPattern\weekly_home_panel_summary.csv', dtype={'census_block_group':str})
# # weekly_residing_df = weekly_residing_df.merge(sociala_variable_df, left_on='census_block_group', right_on='fips')
# weekly_residing_all_df['census_block_group'] = weekly_residing_all_df['census_block_group'].str.zfill(12)
# weekly_residing_all_df = weekly_residing_all_df.sort_values(['date_range_start', 'date_range_end'])
# weekly_residing_all_df

In [2]:
# ACS_csv = r'H:\covid_mobility_results\new_census_data\ACS_2019_5YR_BG\ACS_race_cbsa_income_2019_county_decile.csv'
# ACS_df = pd.read_csv(ACS_csv)

# YEAR = '2022'

# if YEAR > "2019":
#     acs_year = 2019
# else:
#     acs_year = YEAR
    
# social_variable_df = pd.read_csv(fr'P:\Shared drives\T5\safegraph_bias\cbg_acs_{acs_year}_county_tract.csv', dtype={'fips':str})
# social_variable_df['fips'] = social_variable_df['fips'].astype(str).str.zfill(12)
# social_variable_df.columns
# print("US population, male, female:")
# social_variable_df['totalpopulation'].sum(), social_variable_df['male'].sum(), social_variable_df['female'].sum(), len(social_variable_df)

In [114]:
def compose_variables(ACS_df):
    ACS_df['Female'] = ACS_df['female'] / ACS_df['totalpopulation']
    ACS_df['Male'] = ACS_df['male'] / ACS_df['totalpopulation']
    ACS_df['<15'] = (ACS_df['under5years'] + ACS_df['5to9years'] + \
                                        ACS_df['10to14years']) / ACS_df['totalpopulation'] 

    ACS_df['15–17'] = (ACS_df['15to17years']) / ACS_df['totalpopulation'] # ACS_df['under5years'] + ACS_df['5to9years'] + \                                     ACS_df['10to14years'] + 

    ACS_df['18–24'] = ACS_df['18to24years'] / ACS_df['totalpopulation']
    ACS_df['25–34'] = ACS_df['25to34years'] / ACS_df['totalpopulation'] 
    ACS_df['35–44'] = ACS_df['35to44years'] / ACS_df['totalpopulation'] 
    ACS_df['45–54'] = ACS_df['45to54years'] / ACS_df['totalpopulation'] 
    ACS_df['55–64'] = ACS_df['55to64years'] / ACS_df['totalpopulation'] 
    ACS_df['>65'] = (ACS_df['65to74years'] + ACS_df['65to74years'] + ACS_df['75to84years'] \
                                           + ACS_df['85yearsandover']) / ACS_df['totalpopulation']

    ACS_df['White'] = ACS_df['whitealone'] / ACS_df['totalpopulation'] 
    ACS_df['Black'] = ACS_df['blackorafricanamericanalone'] / ACS_df['totalpopulation'] 
    ACS_df['Hispanic'] = ACS_df['Hispanic_or_Latino'] / ACS_df['totalpopulation'] 
    ACS_df['Asian'] = ACS_df['asianalone'] / ACS_df['totalpopulation']
    # Hispanic/Latino
    
    #  # ACS_df['population25yearsandover'] was change to  ACS_df['totalpopulation'] 
    ACS_df['NoSchool'] = ACS_df['population25yearsandoverlessthanhighschool'] / ACS_df['totalpopulation']   # ACS_df['population25yearsandover']
    ACS_df['HighSchool'] = ACS_df['population25yearsandoverhighschoolgraduateincludesequivalency'] / ACS_df['totalpopulation']    # ACS_df['population25yearsandover']
    ACS_df['NoCollege'] = (ACS_df['population25yearsandoverlessthanhighschool'] + \
                                                ACS_df['population25yearsandoverhighschoolgraduateincludesequivalency']) \
                                                / ACS_df['totalpopulation']   # ACS_df['population25yearsandover']

    ACS_df['Bachelor'] = ACS_df['population25yearsandoverbachelorsdegree'] / ACS_df['totalpopulation']    # ACS_df['population25yearsandover']
    ACS_df['Graduate'] = (ACS_df['population25yearsandovermastersdegree'] + ACS_df['population25yearsandoverprofessionalschooldegree'] + \
                                                   ACS_df['population25yearsandoverdoctoratedegree']) / ACS_df['totalpopulation'] 
      # ACS_df['population25yearsandover']
    
    
    # convert household to population
    ACS_df['household_size'] = ACS_df['totalpopulation']  / ACS_df['households']
    ACS_df['<20K'] = ACS_df['household_size'] * (ACS_df['householdslessthan10000'] + ACS_df['households10000to14999'] + \
                                                   ACS_df['households15000to19999']) / ACS_df['totalpopulation']   
    ACS_df['20K–30K'] = ACS_df['household_size'] * (ACS_df['households20000to24999'] + ACS_df['households25000to29999']) / ACS_df['totalpopulation']  
    ACS_df['30K–50K'] = ACS_df['household_size'] * (ACS_df['households30000to34999'] + ACS_df['households35000to39999'] + \
                                           ACS_df['households40000to44999'] + ACS_df['households45000to49999']) / ACS_df['totalpopulation']
    ACS_df['50K–100K'] = ACS_df['household_size'] * (ACS_df['households50000to59999'] + ACS_df['households60000to74999'] + \
                                            ACS_df['households75000to99999']) / ACS_df['totalpopulation']   # 

    ACS_df['>100K'] = ACS_df['household_size'] * (ACS_df['households100000to124999'] + ACS_df['households125000to149999'] + \
                                            ACS_df['households150000to199999'] + ACS_df['households200000ormore']) / ACS_df['totalpopulation']

    ACS_df['fips'] = ACS_df['fips'].astype(str).str.zfill(12)
    
    
    ACS_df['cbg_fips'] = ACS_df['fips'] 
    ACS_df['tract_fips'] = ACS_df['cbg_fips'].str[:11]
    ACS_df['county_fips'] = ACS_df['cbg_fips'].str[:5]
    ACS_df['state_fips'] = ACS_df['cbg_fips'].str[:2]
    
    ACS_df['nation_fips'] = 0
    ACS_df['nation_fips'] = ACS_df['nation_fips'].astype(str)
    
    # print(ACS_df[columns].describe())
    # print(len(ACS_df))
    
    return ACS_df    

In [115]:
# load monthly or weekly device sampling data


data_dir = r'L:\SafeGraph_monthly_patterns_2018-2022'
save_dir = r'P:\Shared drives\T5\safegraph_bias\refined_20230309'
svae_path = save_dir

os.makedirs(save_dir, exist_ok=True)

years = [2018, 2019, 2020, 2021, 2022]

columns = ['Female', 'Male',
       '<15', '15–17', '18–24', '25–34', '35–44', '45–54', '55–64', '>65',
       'White', 'Black', 'Hispanic', 'Asian', 'NoSchool', 'HighSchool',
       'NoCollege', 'Bachelor', 'Graduate', '<20K', '20K–30K', '30K–50K',
       '50K–100K', '>100K']

def load_data(year):
    file_name = os.path.join(data_dir, str(year), f'US_monthly_home_panel_summary_{year}.csv')
    try:
        residing_df = pd.read_csv(file_name, dtype={'census_block_group':str}, encoding='utf16')
    except:
        residing_df = pd.read_csv(file_name, dtype={'census_block_group':str}, encoding='utf8')
    residing_df['census_block_group'] = residing_df['census_block_group'].astype(str).str.zfill(12)
    
    residing_df['date_range_start'] = residing_df['year'].astype(str).str.zfill(4)  + residing_df['month'].astype(str).str.zfill(2) 
    
    if year > 2019:
        acs_year = 2019
    else:
        acs_year = year

    social_variable_df = pd.read_csv(fr'P:\Shared drives\T5\safegraph_bias\cbg_acs_{acs_year}_county_tract.csv', 
                                     dtype={'fips':str, 'tract_fips':str, 'totalpopulation':int})
    social_variable_df['fips'] = social_variable_df['fips'].astype(str).str.zfill(12)
    
    print(f"US population: {social_variable_df['totalpopulation'].sum()}, male, female:", 
          social_variable_df['female'].sum(), social_variable_df['male'].sum(), '\n')
    # social_variable_df['totalpopulation'].sum(), social_variable_df['male'].sum(), social_variable_df['female'].sum(), len(social_variable_df)
    social_variable_df = compose_variables(ACS_df=social_variable_df)
    
    # return social_variable_df
    
    fips_SafeGraph = residing_df['census_block_group'].unique()
    fips_ACS = social_variable_df['fips'].unique()
    
    print(f"FIPS count in SafeGraph：{len(fips_SafeGraph)}, in ACS: {len(fips_ACS)}", '\n')
    
    
    diff1 = sorted(set(fips_SafeGraph) - set(fips_ACS))
    print(f"FIPS only in SafeGraph but NOT in ACS:, {len(diff1)}", diff1, '\n')
    
    # print()
    diff2 = sorted(set(fips_ACS) - set(fips_SafeGraph))
    print(f"FIPS only in ACS but NOT in SafeGraph:, {len(diff2)}", diff2, '\n')
    discarded_ACS_df = social_variable_df[social_variable_df['fips'].isin(diff2)]
    extra_residing_df = residing_df[residing_df['census_block_group'].isin(diff1)]
    
    print(f"Removed devices only in {len(diff1)} SafeGraph FIPS: {extra_residing_df['number_devices_residing'].sum()} devices.", '\n')
    
    print(f"Removed these {len(discarded_ACS_df) } block groups, {discarded_ACS_df['totalpopulation'].sum()} people.", '\n')
    # social_variable_df = social_variable_df[~social_variable_df['fips'].isin(diff2)]
    
    print("Before merge ACS data row count:", len(residing_df), '\n')    
    residing_merged_df = residing_df.merge(social_variable_df, 
                                           left_on='census_block_group', 
                                           right_on='cbg_fips', how='inner')#.fillna(0)  # 'how' is very important to keep all devices and population !
    print("After merge ACS data row count: ", len(residing_merged_df), '\n')
    
    assert residing_merged_df['census_block_group'].isin(diff1).sum() == 0, 'The CBGs only in SafeGraph are not removed! '
    assert residing_merged_df['cbg_fips'].isin(diff2).sum() == 0, 'The CBGs only in ACS are not removed! '
   
    print("ACS columns:")
    print(social_variable_df.columns)

    return residing_merged_df, social_variable_df

# Note: the nan value will be ignored! 
# Note: CBGs have popultion=0 will have nan in population ratio.

# test    
year = 2022
residing_variable_df, ACS_df = load_data(year = year)
# residing_variable_df
residing_variable_df

US population: 327939105, male, female: 166551005 161465237 

FIPS count in SafeGraph：220786, in ACS: 220333 

FIPS only in SafeGraph but NOT in ACS:, 516 ['0CA:13020068', '0CA:13020069', '0CA:13020071', '0CA:13020079', '0CA:13020085', '0CA:13020090', '0CA:13020091', '0CA:13020092', '0CA:13020096', '0CA:13100277', '0CA:13130093', '0CA:13130094', '0CA:13130095', '0CA:13130096', '0CA:13130100', '0CA:13130101', '0CA:13130102', '0CA:13130103', '0CA:13130105', '0CA:13130106', '0CA:13130115', '0CA:13130119', '0CA:13130132', '0CA:24170059', '0CA:24170060', '0CA:24170065', '0CA:24290142', '0CA:24440063', '0CA:24450147', '0CA:24450148', '0CA:24450149', '0CA:24450150', '0CA:24690094', '0CA:24690108', '0CA:35010304', '0CA:35010306', '0CA:35010307', '0CA:35010308', '0CA:35010309', '0CA:35010310', '0CA:35010323', '0CA:35010407', '0CA:35010408', '0CA:35070201', '0CA:35070202', '0CA:35070203', '0CA:35070204', '0CA:35070205', '0CA:35070207', '0CA:35070215', '0CA:35070216', '0CA:35070222', '0CA:3507023

,year,month,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,date_range_start,state_fips,county_fips,tract_fips,fips,state,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore,Female,Male,<15,15–17,18–24,25–34,35–44,45–54,55–64,>65,White,Black,Hispanic,Asian,NoSchool,HighSchool,NoCollege,Bachelor,Graduate,household_size,<20K,20K–30K,30K–50K,50K–100K,>100K,cbg_fips,nation_fips
0,2022,1,ak,US,020200002021,211,91,202201,02,02020,02020000202,020200002021,ak,1189,585,604,89,76,22,41,127,127,86,127,116,117,80,39,1074,0,5,60,0,50,0,0,824,99,175,127,127,62,0,0,616,19,83,59,0,18,72,35,0,21,13,127,36,0,72,33,19,0.50799,0.49201,0.157275,0.034483,0.106812,0.106812,0.07233,0.106812,0.097561,0.296888,0.90328,0.0,0.0,0.050463,0.083263,0.147183,0.230446,0.106812,0.052145,1.930195,0.261364,0.029221,0.207792,0.285714,0.201299,020200002021,0
1,2022,2,ak,US,020200002021,198,95,202202,02,02020,02020000202,020200002021,ak,1189,585,604,89,76,22,41,127,127,86,127,116,117,80,39,1074,0,5,60,0,50,0,0,824,99,175,127,127,62,0,0,616,19,83,59,0,18,72,35,0,21,13,127,36,0,72,33,19,0.50799,0.49201,0.157275,0.034483,0.106812,0.106812,0.07233,0.106812,0.097561,0.296888,0.90328,0.0,0.0,0.050463,0.083263,0.147183,0.230446,0.106812,0.052145,1.930195,0.261364,0.029221,0.207792,0.285714,0.201299,020200002021,0
2,2022,3,ak,US,020200002021,259,184,202203,02,02020,02020000202,020200002021,ak,1189,585,604,89,76,22,41,127,127,86,127,116,117,80,39,1074,0,5,60,0,50,0,0,824,99,175,127,127,62,0,0,616,19,83,59,0,18,72,35,0,21,13,127,36,0,72,33,19,0.50799,0.49201,0.157275,0.034483,0.106812,0.106812,0.07233,0.106812,0.097561,0.296888,0.90328,0.0,0.0,0.050463,0.083263,0.147183,0.230446,0.106812,0.052145,1.930195,0.261364,0.029221,0.207792,0.285714,0.201299,020200002021,0
3,2022,4,ak,US,020200002021,222,231,202204,02,02020,02020000202,020200002021,ak,1189,585,604,89,76,22,41,127,127,86,127,116,117,80,39,1074,0,5,60,0,50,0,0,824,99,175,127,127,62,0,0,616,19,83,59,0,18,72,35,0,21,13,127,36,0,72,33,19,0.50799,0.49201,0.157275,0.034483,0.106812,0.106812,0.07233,0.106812,0.097561,0.296888,0.90328,0.0,0.0,0.050463,0.083263,0.147183,0.230446,0.106812,0.052145,1.930195,0.261364,0.029221,0.207792,0.285714,0.201299,020200002021,0
4,2022,5,ak,US,020200002021,281,260,202205,02,02020,02020000202,020200002021,ak,1189,585,604,89,76,22,41,127,127,86,127,116,117,80,39,1074,0,5,60,0,50,0,0,824,99,175,127,127,62,0,0,616,19,83,59,0,18,72,35,0,21,13,127,36,0,72,33,19,0.50799,0.49201,0.157275,0.034483,0.106812,0.106812,0.07233,0.106812,0.097561,0.296888,0.90328,0.0,0.0,0.050463,0.083263,0.147183,0.230446,0.106812,0.052145,1.930195,0.261364,0.029221,0.207792,0.285714,0.201299,020200002021,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [116]:
# residing_variable_df

In [118]:
residing_variable_df[columns].describe()

,Female,Male,<15,15–17,18–24,25–34,35–44,45–54,55–64,>65,White,Black,Hispanic,Asian,NoSchool,HighSchool,NoCollege,Bachelor,Graduate,<20K,20K–30K,30K–50K,50K–100K,>100K
count,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.629960e+06,2.624514e+06,2.624514e+06,2.624514e+06,2.624514e+06,2.624514e+06
mean,5.090498e-01,4.909609e-01,1.538363e-01,3.518153e-02,6.514063e-02,8.569376e-02,8.432689e-02,8.874065e-02,9.290039e-02,2.207146e-01,7.296496e-01,4.841134e-02,1.719117e-01,2.552590e-02,6.192621e-02,1.951605e-01,2.570867e-01,7.843160e-02,7.065565e-02,1.582671e-01,9.352759e-02,1.723093e-01,2.826985e-01,2.581354e-01
std,6.573017e-02,6.573035e-02,6.534775e-02,2.568942e-02,3.847744e-02,4.413563e-02,3.943278e-02,4.212652e-02,4.699138e-02,1.177144e-01,2.660630e-01,6.423698e-02,2.405636e-01,3.778517e-02,4.693026e-02,9.693194e-02,1.202853e-01,4.658197e-02,6.536729e-02,1.318307e-01,7.455660e-02,9.775678e-02,1.131847e-01,1.800206e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.744624e-01,4.549214e-01,1.134806e-01,1.658925e-02,4.095563e-02,5.629433e-02,5.791154e-02,6.001890e-02,6.079464e-02,1.417551e-01,5.961252e-01,0.000000e+00,1.424121e-02,0.000000e+00,2.791878e-02,1.253731e-01,1.727067e-01,4.727273e-02,2.395210e-02,6.067961e-02,3.767123e-02,1.011342e-01,2.075000e-01,1.153846e-01
50%,5.088435e-01,4.911591e-01,1.524664e-01,3.286385e-02,6.060606e-02,7.967378e-02,8.017677e-02,8.349770e-02,8.576814e-02,2.056075e-01,8.232044e-01,2.773826e-02,6.686879e-02,6.987578e-03,5.477981e-02,1.907810e-01,2.556719e-01,7.154930e-02,5.330073e-02,1.255061e-01,8.023863e-02,1.643836e-01,2.820513e-01,2.266910e-01
75%,5.450807e-01,5.255392e-01,1.932520e-01,4.967427e-02,8.421751e-02,1.082090e-01,1.060100e-01,1.117381e-01,1.169429e-01,2.808483e-01,9.394387e-01,7.344262e-02,2.181122e-01,4.103243e-02,8.581081e-02,2.588117e-01,3.364055e-01,1.021505e-01,9.822485e-02,2.224231e-01,1.338583e-01,2.316514e-01,3.550088e-01,3.763441e-01
max,1.000000e+00,1.000000e+00,7.676768e-01,8.481013e-01,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [107]:
residing_variable_df['number_devices_residing'].isna().sum()
residing_variable_df.isna().sum()

Unnamed: 0              0
year                    0
month                   0
region                  0
iso_country_code        0
                    ...  
30K–50K             15607
50K–100K            15607
>100K               15607
cbg_fips                0
nation_fips             0
Length: 89, dtype: int64

In [ ]:
# residing_merged_df['Female'].mean(), residing_merged_df['Asian'].mean(), residing_merged_df['White'].mean(), residing_merged_df['Black'].mean(), residing_merged_df['Hispanic'].mean(), residing_merged_df['50K–100K'].mean()

In [52]:
# residing_merged_df['Female'].mean(), residing_merged_df['Asian'].mean(), residing_merged_df['White'].mean(), residing_merged_df['Black'].mean(), residing_merged_df['Hispanic'].mean(), residing_merged_df['50K–100K'].mean()

In [80]:
# Issue: the income range cannot sum to 1
# residing_variable_df.sample(10).iloc[:, -5:].sum(axis=1)

# Compute device sampling rate

## Monthly

In [73]:
import numpy as np
def get_sampling_rate(residing_df, ACS_df, aggre_columns):
    # residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]
    aggre_columns = ['date_range_start'] + aggre_columns
    
    # device_count_total = # including have
    
    sampling_rate_df = residing_df.groupby(aggre_columns).agg(
                                      
                                      start_date=('date_range_start', 'first'), 
                                      end_date=('date_range_start', 'last'), 
                                      # period_count=('CBG_count', 'count'), 
                                      CBG_count=('totalpopulation', 'count'), 
                                      fips=(aggre_columns[-1], 'first'), 
                                      device_count=('number_devices_residing', 'sum'), 
                                      population=('totalpopulation', 'sum'), 
                                      year=('year', 'first'),
                                      aggre_column_value=(aggre_columns[-1], 'first'), 
                                      # safegraph_ratio_max=('safegraph_ratio', max),
                                      # census_ratio=('census_ratio', 'mean'),
                                      # census_ratio_max=('census_ratio', max),
                                      # census_ratio_min=('census_ratio', min),
                                     )
    
    # sampling_rate_df['fips'] = 
    
    sampling_rate_df['population'] = sampling_rate_df['population'].astype(int)
    
    if aggre_columns[-1] == 'nation_fips':
        sampling_rate_df['population'] = ACS_df['totalpopulation'].sum()
        # sampling_rate_df['population'] = ACS_df['totalpopulation'].sum()
    
    sampling_rate_df['sampling_rate'] = sampling_rate_df['device_count'] / sampling_rate_df['population']
    sampling_rate_df.replace([np.inf], np.nan, inplace=True)

    return sampling_rate_df.reset_index().iloc[:, 2:]

  
def compute_sampling_rates(df, ACS_df, aggre_columns = ['nation_fips', 'state_fips', 'county_fips', 'tract_fips', 'cbg_fips']):
    df_list = []
    for c in aggre_columns:
        sampling_rate_df = get_sampling_rate(df, ACS_df, aggre_columns=[c])
        df_list.append(sampling_rate_df)
        
    df_all = pd.concat(df_list)
    
    return df_all

# df_all = compute_sampling_rates(residing_variable_df, aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])
# df_all = compute_sampling_rates(df,                        aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])
df_all = compute_sampling_rates(residing_variable_df, ACS_df=ACS_df, aggre_columns = ['nation_fips', 'state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])

df_all['fips_len'] = df_all['fips'].astype(str).str.len()
df_all['fips_level'] = df_all['fips_len'].astype(str)
df_all['fips_level'] = df_all['fips_level'].replace({'1': 'nation', '2':"state", '5':"county", '11': 'tract', '12':'CBG'})

sampling_rates_csv_name = fr'P:\Shared drives\T5\safegraph_bias\sampling_rates_{year}_monthly.csv'
df_all.to_csv(sampling_rates_csv_name, index=False)
df_all

,start_date,end_date,CBG_count,fips,device_count,population,year,aggre_column_value,sampling_rate,fips_len,fips_level
0,201801,201801,219981,0,17484627,326228729,2018,0,0.053596,1,nation
1,201802,201802,219933,0,16778854,326228729,2018,0,0.051433,1,nation
2,201803,201803,220007,0,22779282,326228729,2018,0,0.069826,1,nation
3,201804,201804,219944,0,24292409,326228729,2018,0,0.074464,1,nation
4,201805,201805,219945,0,27432594,326228729,2018,0,0.084090,1,nation
...,...,...,...,...,...,...,...,...,...,...,...
2639492,201812,201812,1,721537506011,25,699,2018,721537506011,0.035765,12,CBG
2639493,201812,201812,1,721537506012,84,2432,2018,721537506012,0.034539,12,CBG
2639494,201812,201812,1,721537506013,37,976,2018,721537506013,0.037910,12,CBG
2639495,201812,201812,1,721537506021,28,1707,2018,721537506021,0.016403,12,CBG


In [74]:
df_all['fips_level'].value_counts()

CBG       2639497
tract      885337
county      38640
state         624
nation         12
Name: fips_level, dtype: int64

In [75]:
df_all.groupby(['fips_level']).mean()

C:\Users\N\AppData\Local\Temp\ipykernel_26404\2664521766.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_all.groupby(['fips_level']).mean()


,CBG_count,device_count,population,year,sampling_rate,fips_len
fips_level,,,,,,
CBG,1.000000,1.065427e+02,1.483065e+03,2018.0,0.079729,12.0
county,68.309964,7.277927e+03,1.013081e+05,2018.0,0.064119,5.0
nation,219958.083333,2.343492e+07,3.262287e+08,2018.0,0.071836,1.0
state,4229.963141,4.506716e+05,6.273311e+06,2018.0,0.068860,2.0
tract,2.981347,3.176407e+02,4.421532e+03,2018.0,0.085941,11.0


## Aggregate to yearly

In [76]:
def time_aggregate(sampling_rate_df):
    
    rate_gb_df = sampling_rate_df.groupby(['year', 'fips']).agg(
                                      start_date=('start_date', 'first'), 
                                      end_date=('end_date', 'last'), 
                                      period_count=('start_date', 'count'), 
                                      CBG_count=('CBG_count', 'mean'), 
                                      device_count_mean=('device_count', 'mean'), 
                                      population=('population', 'mean'), 
                                      sampling_rate_mean=('sampling_rate', 'mean'),
                                      sampling_rate_max=('sampling_rate', max),
                                      sampling_rate_min=('sampling_rate', min),        
                                     )
    
    return rate_gb_df
    
rate_gb_df = time_aggregate(df_all.iloc[:]).reset_index()# .query(" sampling_rate < 0.5 ")).reset_index()  # remove the outliers
# rate_gb_df.to_csv(r'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_yearly.csv', index=False)
rate_gb_df['fips_len'] = rate_gb_df['fips'].astype(str).str.len()
rate_gb_df['fips_level'] = rate_gb_df['fips_len'].astype(str)
rate_gb_df['fips_level'] = rate_gb_df['fips_level'].replace({'1':'nation', '2':"state", '5':"county", '11': 'tract', '12':'cbg'})

rate_gb_df.to_csv(fr'P:\Shared drives\T5\safegraph_bias\sampling_rates_{year}_monthly_yearly.csv', index=False)
# 
rate_gb_df

,year,fips,start_date,end_date,period_count,CBG_count,device_count_mean,population,sampling_rate_mean,sampling_rate_max,sampling_rate_min,fips_len,fips_level
0,2018,0,201801,201812,12,219958.083333,2.343492e+07,326228729.0,0.071836,0.084090,0.051433,1,nation
1,2018,01,201801,201812,12,3437.750000,4.025513e+05,4864680.0,0.082750,0.093259,0.063297,2,state
2,2018,01001,201801,201812,12,32.000000,4.349000e+03,55200.0,0.078786,0.089764,0.055145,5,county
3,2018,01001020100,201801,201812,12,2.000000,1.511667e+02,1923.0,0.078610,0.120645,0.042642,11,tract
4,2018,010010201001,201801,201812,12,1.000000,4.758333e+01,636.0,0.074817,0.196541,0.033019,12,cbg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
297396,2018,721537506012,201801,201812,12,1.000000,7.691667e+01,2432.0,0.031627,0.039885,0.016859,12,cbg
297397,2018,721537506013,201801,201812,12,1.000000,3.200000e+01,976.0,0.032787,0.044057,0.024590,12,cbg
297398,2018,72153750602,201801,201812,12,2.000000,5.066667e+01,2511.0,0.020178,0.029072,0.006372,11,tract
297399,2018,721537506021,201801,201812,12,1.000000,2.716667e+01,1707.0,0.015915,0.023433,0.006444,12,cbg


# Compute bias

In [95]:
from tqdm import tqdm
from datetime import datetime
import os
tqdm.pandas()

columns = ['Female', 'Male',
       '<15', '15–17', '18–24', '25–34', '35–44', '45–54', '55–64', '>65',
       'White', 'Black', 'Hispanic', 'Asian', 'NoSchool', 'HighSchool',
       'NoCollege', 'Bachelor', 'Graduate', '<20K', '20K–30K', '30K–50K',
       '50K–100K', '>100K']

used_columns = ['date_range_start', 'nation_fips', 'state_fips', 'county_fips', 'tract_fips', 'number_devices_residing', 'totalpopulation'] + columns

def get_bias(df, aggre_column):
    
    # if df.isna().sum() > 0:
    #     print(df)
#     now = datetime.now()
#     current_time = now.strftime("%H:%M:%S")
#     if current_time[-2] == '5':
#         print("Current state:", df.iloc[0]['state_fips'])
        
#     if current_time[-2] == '2':
#         print("Current state:", df.iloc[0]['state_fips'])
    
    df = df.dropna()
    
    aggre_total_device = df['number_devices_residing'].sum()
    
    aggre_total_pop = df['totalpopulation'].sum()
    
    if  aggre_column == 'nation_fips':
        aggre_total_pop = ACS_df['totalpopulation'].sum()
#         print(f"Use the US population {aggre_total_pop} the the naion level bias:")
    
    result_df = pd.DataFrame(columns = [ 'start_date', 'census_ratio', 'safegraph_ratio', 'CBG_count', 'device_count', 'populaion', 'safegraph_ratio_to_census', 'bias'],
                   index = columns)
    
    result_df.index.name = 'variable'
    
    
    # return result_df
    for column in columns[:]:
        # print(column)
        start_date = df.iloc[0]['date_range_start'][:10] 
        
        # df[column] = df[column] / df['totalpopulation']  # population group ratio of each CBG
        
        cbg_column_devices = df['number_devices_residing'] * df[column]   # population group device count  of each CBG
        # cbg_device_ratio = cbg_column_devices.sum() / state_total_device  # aggregated population group device count / total_device

        cbg_column_pop = df[column] * df['totalpopulation']  # group population  of each CBG        
        census_ratio = cbg_column_pop.sum() / aggre_total_pop   # group population ratio (to the aggregated population)    
        
        safegraph_ratio = cbg_column_devices.sum() / aggre_total_device   # group population device ratio (to the aggregated device count)    
        
        # result_df.loc[column]['variable'] = column
        result_df.loc[column]['start_date'] = start_date
        result_df.loc[column]['CBG_count'] = len(df)
        result_df.loc[column]['census_ratio'] = census_ratio
        result_df.loc[column]['safegraph_ratio'] = safegraph_ratio
        result_df.loc[column]['populaion'] = cbg_column_pop.sum()
        result_df.loc[column]['device_count'] = cbg_column_devices.sum() 
        result_df.loc[column]['safegraph_ratio_to_census'] = result_df.loc[column]['safegraph_ratio'] / result_df.loc[column]['census_ratio'] 
        result_df.loc[column]['bias'] = result_df.loc[column]['safegraph_ratio_to_census'] - 1
 

    return result_df


def get_aggregated_bias(residing_df, aggre_columns):
    
    residing_df.loc[:, 'state_fips'] = residing_df['state_fips'].astype(str).str.zfill(2)
    
    residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]   # have an issue of different starting hours 
    groupby_cols = ['date_range_start'] + aggre_columns
    # return aggre_columns

    bias_df = residing_df.sort_values('date_range_start').groupby(groupby_cols).progress_apply(get_bias, aggre_column=aggre_columns[-1])
    
    # return bias_df

    bias_df.reset_index(drop=False, inplace=True)
    # bias_df['start_date'] = bias_df['date_range_start'].astype(str).str[:10] 
    # bias_df = bias_df.drop(columns=['date_range_start'])
    
    # return aggre_columns + ['variable']
    
    # yearly
    bias_gb_df = bias_df.groupby(aggre_columns + ['variable']).agg(
                                      start_date=('start_date', 'first'), 
                                      end_date=('start_date', 'last'), 
                                      period_count=('CBG_count', 'count'), 
                                      CBG_count=('CBG_count', 'mean'), 
                                      safegraph_ratio_mean=('safegraph_ratio', 'mean'), 
                                      safegraph_ratio_min=('safegraph_ratio', min), 
                                      safegraph_ratio_max=('safegraph_ratio', max),
                                      census_ratio=('census_ratio', 'mean'),
                                      device_count=('device_count', 'mean'),
                                      population=('populaion', 'mean'),
                                     )
    
    # return bias_gb_df

    # if len(aggre_columns) == 1:
    #     bias_gb_df = bias_gb_df.loc[:, columns, :]  
    # if len(aggre_columns) == 2:
    #     bias_gb_df = bias_gb_df.loc[:, :, columns, :]  
    # if len(aggre_columns) == 3:
    #     bias_gb_df = bias_gb_df.loc[:, :, :, columns, :]  

    bias_gb_df['safegraph_ratio_mean_to_census'] = bias_gb_df['safegraph_ratio_mean'] / bias_gb_df['census_ratio']
    bias_gb_df['safegraph_ratio_min_to_census' ] = bias_gb_df['safegraph_ratio_min' ] / bias_gb_df['census_ratio']
    bias_gb_df['safegraph_ratio_max_to_census' ] = bias_gb_df['safegraph_ratio_max' ] / bias_gb_df['census_ratio']
    bias_gb_df['bias' ] = bias_gb_df['safegraph_ratio_mean_to_census' ] - 1
    
    bias_gb_df['CBG_count'] = bias_gb_df['CBG_count'].astype(int)
    
    bias_gb_df = bias_gb_df.reset_index()
    bias_gb_df['variable'] = pd.Categorical(bias_gb_df['variable'], categories=columns, ordered=True)

    bias_gb_df = bias_gb_df.sort_values(aggre_columns + ['variable']) #   sort

  
    return bias_gb_df, bias_df

def compute_nation_bias():
    # save_path = r'P:\Shared drives\T5\safegraph_bias'
    new_name = os.path.join(save_dir, f'monthly_bias_nation_{year}.csv')

    aggre_columns = ['nation_fips']
    
    if  aggre_columns[-1] == 'nation_fips':
    # aggre_total_pop = ACS_df['totalpopulation'].sum()
        print(f"Use the US population {ACS_df['totalpopulation'].sum() } for the naion level bias:")

    bias_gb_df, bias_df = get_aggregated_bias(residing_df=residing_variable_df.dropna(), aggre_columns=aggre_columns[:])
    # bias_gb_df = get_aggregated_bias(residing_df=SC_residing_df[used_columns], aggre_columns=aggre_columns[:])
    bias_gb_df.to_csv(new_name, index=False)
    
    new_name = os.path.join(save_dir, f'monthly_bias_nation_{year}_time.csv')
    bias_df.to_csv(new_name, index=False)
    
    return bias_gb_df, bias_df

def compute_state_bias():
    # save_path = r'P:\Shared drives\T5\safegraph_bias'
    new_name = os.path.join(save_dir, f'monthly_bias_state_{year}.csv')
    aggre_columns = ['state_fips']

    # bias_gb_df = get_aggregated_bias(residing_df=SC_residing_df[used_columns], aggre_columns=aggre_columns)
    bias_gb_df, bias_df = get_aggregated_bias(residing_df=residing_variable_df[used_columns].dropna(), aggre_columns=aggre_columns)
    
    bias_gb_df.to_csv(new_name, index=False)
    
    new_name = os.path.join(save_dir, f'monthly_bias_state_{year}_time.csv')
    bias_df.to_csv(new_name, index=False)
    
    return bias_gb_df, bias_df

def compute_county_bias():
    # save_path = r'P:\Shared drives\T5\safegraph_bias'
    new_name = os.path.join(save_dir, f'monthly_bias_county_{year}.csv')

    aggre_columns = ['state_fips', 'county_fips']
    bias_gb_df, bias_df = get_aggregated_bias(residing_df=residing_variable_df.iloc[:].dropna(), aggre_columns=aggre_columns[:])
    # bias_gb_df = get_aggregated_bias(residing_df=SC_residing_df[used_columns], aggre_columns=aggre_columns[:])
    bias_gb_df.to_csv(new_name, index=False)
    
    new_name = os.path.join(save_dir, f'monthly_bias_county_{year}_time.csv')
    bias_df.to_csv(new_name, index=False)
    
    return bias_gb_df, bias_df

# Not used! 
# def compute_tract_bias():
#     save_path = r'P:\Shared drives\T5\safegraph_bias'
#     # new_name = os.path.join(save_path, f'monthly_bias_tract_{YEAR}.csv')

#     aggre_columns = ['state_fips', 'county_fips', 'tract_fips']
#     bias_gb_df, bias_df = get_aggregated_bias(residing_df=residing_variable_df[used_columns].dropna(), aggre_columns=aggre_columns[:])
#     # bias_gb_df = get_aggregated_bias(residing_df=SC_residing_df.iloc[:1000], aggre_columns=aggre_columns[:])
    
#     bias_gb_df.to_csv(new_name, index=False)
    
#     # new_name = os.path.join(save_path, f'monthly_bias_tract_{YEAR}_time.csv')
#     bias_df.to_csv(new_name, index=False)
    
#     return bias_gb_df, bias_df
# bias_gb_df, bias_df = compute_tract_bias()

# YEAR = year

# bias_gb_df, bias_df = compute_nation_bias()
# bias_gb_df, bias_df = compute_state_bias()
# bias_gb_df, bias_df = compute_county_bias()


bias_gb_df.round(3)   

,state_fips,variable,start_date,end_date,period_count,CBG_count,safegraph_ratio_mean,safegraph_ratio_min,safegraph_ratio_max,census_ratio,device_count,population,safegraph_ratio_mean_to_census,safegraph_ratio_min_to_census,safegraph_ratio_max_to_census,bias
16,01,Female,202201,202212,12,3431,0.517,0.516,0.520,0.516,303141.771,NaN,1.002,0.999,1.007,0.002
20,01,Male,202201,202212,12,3431,0.483,0.480,0.484,0.484,282323.313,NaN,0.997,0.993,1.001,-0.003
9,01,<15,202201,202212,12,3431,0.143,0.142,0.145,0.146,84023.214,NaN,0.983,0.973,0.993,-0.017
0,01,15–17,202201,202212,12,3431,0.035,0.035,0.035,0.035,20387.057,NaN,0.990,0.983,1.004,-0.010
1,01,18–24,202201,202212,12,3431,0.059,0.059,0.060,0.060,34747.551,NaN,0.996,0.984,1.007,-0.004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1234,72,<20K,202201,202212,12,2504,0.374,0.360,0.385,0.407,21585.165,NaN,0.918,0.885,0.946,-0.082
1226,72,20K–30K,202201,202212,12,2504,0.137,0.131,0.140,0.146,7913.514,NaN,0.936,0.899,0.960,-0.064
1228,72,30K–50K,202201,202212,12,2504,0.177,0.171,0.182,0.175,10194.821,NaN,1.010,0.974,1.038,0.010
1231,72,50K–100K,202201,202212,12,2504,0.168,0.164,0.175,0.140,9631.817,NaN,1.201,1.175,1.249,0.201


In [429]:
# bias_gb_df.to_csv(r'P:\Shared drives\NJIT_T5\safegraph_bias\weekly_bias_county_2021.csv', index=False)

In [ ]:
stop

# All code

In [96]:
for year in [2018, 2019, 2020, 2021, 2022][::-1]:
    print(year)
    if year > 2019:
        acs_year = 2019
    else:
        acs_year = year
 

    residing_variable_df, ACS_df = load_data(year = year)
    df_all = compute_sampling_rates(residing_variable_df, ACS_df=ACS_df, aggre_columns = ['nation_fips', 'state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])

    df_all['fips_len'] = df_all['fips'].astype(str).str.len()
    df_all['fips_level'] = df_all['fips_len'].astype(str)
    df_all['fips_level'] = df_all['fips_level'].replace({'1': 'nation', '2':"state", '5':"county", '11': 'tract', '12':'CBG'})

    sampling_rates_csv_name = os.path.join(save_dir, fr'sampling_rates_{year}_monthly.csv')
    df_all.to_csv(sampling_rates_csv_name, index=False)
    
    rate_gb_df = time_aggregate(df_all.iloc[:]).reset_index()# .query(" sampling_rate < 0.5 ")).reset_index()  # remove the outliers
    # rate_gb_df.to_csv(r'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_yearly.csv', index=False)
    rate_gb_df['fips_len'] = rate_gb_df['fips'].astype(str).str.len()
    rate_gb_df['fips_level'] = rate_gb_df['fips_len'].astype(str)
    rate_gb_df['fips_level'] = rate_gb_df['fips_level'].replace({'1':'nation', '2':"state", '5':"county", '11': 'tract', '12':'cbg'})

    rate_gb_df.to_csv(os.path.join(save_dir, fr'sampling_rates_{year}_monthly_yearly.csv'), index=False)
    # 
    # bias
    
    bias_gb_df, bias_df = compute_nation_bias()
    bias_gb_df, bias_df = compute_state_bias()
    bias_gb_df, bias_df = compute_county_bias()
    

2022
US population: 327939105, male, female: 166551005 161465237 

FIPS count in SafeGraph：220786, in ACS: 220333 

FIPS only in SafeGraph but NOT in ACS:, 516 ['0CA:13020068', '0CA:13020069', '0CA:13020071', '0CA:13020079', '0CA:13020085', '0CA:13020090', '0CA:13020091', '0CA:13020092', '0CA:13020096', '0CA:13100277', '0CA:13130093', '0CA:13130094', '0CA:13130095', '0CA:13130096', '0CA:13130100', '0CA:13130101', '0CA:13130102', '0CA:13130103', '0CA:13130105', '0CA:13130106', '0CA:13130115', '0CA:13130119', '0CA:13130132', '0CA:24170059', '0CA:24170060', '0CA:24170065', '0CA:24290142', '0CA:24440063', '0CA:24450147', '0CA:24450148', '0CA:24450149', '0CA:24450150', '0CA:24690094', '0CA:24690108', '0CA:35010304', '0CA:35010306', '0CA:35010307', '0CA:35010308', '0CA:35010309', '0CA:35010310', '0CA:35010323', '0CA:35010407', '0CA:35010408', '0CA:35070201', '0CA:35070202', '0CA:35070203', '0CA:35070204', '0CA:35070205', '0CA:35070207', '0CA:35070215', '0CA:35070216', '0CA:35070222', '0CA:35

C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df.loc[:, 'state_fips'] = residing_df['state_fips'].astype(str).str.zfill(2)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]   # have an issue of different starting hours
100%|████████████████████████████████████████████████████████████████████

2021
US population: 327939105, male, female: 166551005 161465237 

FIPS count in SafeGraph：220534, in ACS: 220333 

FIPS only in SafeGraph but NOT in ACS:, 345 ['0CA:24450148', '600109506002', '600109507002', '600109507003', '600109509002', '600109509003', '600109509004', '600509510002', '600509511001', '600509511002', '600509512011', '600509512012', '600509512021', '600509512032', '600509512033', '600509513002', '600509515001', '600509516004', '660109501001', '660109501002', '660109501003', '660109501004', '660109501005', '660109502003', '660109502004', '660109502005', '660109503002', '660109503003', '660109503004', '660109503009', '660109504011', '660109504012', '660109504013', '660109504014', '660109504015', '660109504016', '660109504021', '660109504022', '660109504023', '660109505011', '660109505012', '660109505021', '660109505022', '660109505023', '660109505024', '660109505025', '660109507011', '660109507012', '660109507013', '660109507014', '660109507015', '660109507016', '660109

C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df.loc[:, 'state_fips'] = residing_df['state_fips'].astype(str).str.zfill(2)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]   # have an issue of different starting hours
100%|████████████████████████████████████████████████████████████████████

2020
US population: 327939105, male, female: 166551005 161465237 

FIPS count in SafeGraph：220549, in ACS: 220333 

FIPS only in SafeGraph but NOT in ACS:, 380 ['600109501001', '600109503001', '600109503002', '600109503003', '600109505001', '600109505002', '600109505003', '600109506001', '600109506002', '600109506003', '600109507001', '600109507002', '600109507003', '600109509001', '600109509002', '600109509003', '600109509004', '600509510001', '600509510002', '600509511001', '600509511002', '600509511003', '600509512011', '600509512012', '600509512021', '600509512022', '600509512031', '600509512032', '600509512033', '600509513001', '600509513002', '600509515001', '600509516002', '600509516003', '600509516004', '660109501001', '660109501002', '660109501003', '660109501004', '660109501005', '660109502003', '660109502004', '660109502005', '660109503002', '660109503003', '660109503004', '660109503006', '660109503007', '660109503008', '660109503009', '660109504011', '660109504012', '660109

C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df.loc[:, 'state_fips'] = residing_df['state_fips'].astype(str).str.zfill(2)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]   # have an issue of different starting hours
100%|████████████████████████████████████████████████████████████████████

2019
US population: 327939105, male, female: 166551005 161465237 

FIPS count in SafeGraph：220539, in ACS: 220333 

FIPS only in SafeGraph but NOT in ACS:, 379 ['0CA:24680061', '0CA:59490105', '600109501001', '600109501002', '600109502001', '600109502002', '600109503001', '600109503002', '600109503003', '600109505001', '600109505002', '600109505003', '600109506001', '600109506002', '600109506003', '600109507001', '600109507002', '600109507003', '600109509001', '600109509002', '600109509003', '600109509004', '600209518001', '600209518002', '600509510001', '600509510002', '600509511001', '600509511002', '600509511003', '600509512011', '600509512012', '600509512021', '600509512022', '600509512031', '600509512032', '600509512033', '600509513001', '600509513002', '600509515001', '600509516001', '600509516003', '600509516004', '660109501001', '660109501002', '660109501003', '660109501004', '660109501005', '660109502003', '660109502004', '660109502005', '660109503002', '660109503003', '660109

C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df.loc[:, 'state_fips'] = residing_df['state_fips'].astype(str).str.zfill(2)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]   # have an issue of different starting hours
100%|████████████████████████████████████████████████████████████████████

2018
US population: 326228729, male, female: 165692916 160597055 

FIPS count in SafeGraph：220601, in ACS: 220333 

FIPS only in SafeGraph but NOT in ACS:, 390 ['0CA:24450148', '0CA:24680061', '600109501001', '600109501002', '600109502001', '600109502002', '600109503001', '600109503002', '600109503003', '600109505001', '600109505002', '600109505003', '600109506001', '600109506002', '600109506003', '600109507001', '600109507002', '600109507003', '600109509001', '600109509002', '600109509003', '600109509004', '600209518001', '600509510001', '600509510002', '600509511001', '600509511002', '600509511003', '600509512011', '600509512012', '600509512021', '600509512022', '600509512031', '600509512032', '600509512033', '600509513001', '600509513002', '600509515001', '600509516001', '600509516002', '600509516003', '600509516004', '660109501001', '660109501002', '660109501003', '660109501004', '660109501005', '660109502003', '660109502004', '660109502005', '660109503001', '660109503002', '660109

C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df.loc[:, 'state_fips'] = residing_df['state_fips'].astype(str).str.zfill(2)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\2932647374.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]   # have an issue of different starting hours
100%|████████████████████████████████████████████████████████████████████

# Compute device sampling rate

# Tools

## Merge monthly home panel CSV files 

In [113]:
import os
import pandas as pd



def get_all_files(data_dir, exts=['.csv'], verbose=True):
    all_files = []
    for root_dir, folders, files in os.walk(data_dir):
        # print(root_dir)
        for f in files:
            for ext in exts:
                ext_len = len(ext)
                if ext == f[-ext_len:]:
                    full_name = os.path.join(root_dir, f)
                    all_files.append(full_name)
                    
    if verbose:
        print("Found files:", len(all_files))
    return sorted(all_files)


# csv_files.pop(9)  # Error when downlaoding: 'L:\\SafeGraph_monthly_patterns_2018-2022\\2022\\09\\06\\SAFEGRAPH\\MP\\home_panel_summary.csv',  
# csv_files.pop(11)   # identical to 11-01:  'L:\\SafeGraph_monthly_patterns_2018-2022\\2022\\11\\22\\SAFEGRAPH\\MP\\home_panel_summary.csv',


# YEAR = 2018

years = [2018, 2019, 2020, 2021, 2022]
# years = [2022]

for year in years:
    
    data_dir = fr'L:\SafeGraph_monthly_patterns_2018-2022\{year}'
    
    csv_files = get_all_files(data_dir, exts=['home_panel_summary.csv'])

    
    print("Year:", year)

    df_list = []
    for f in csv_files:
        sep = '\\'  # path separate for windows 
        first_day = f.split(sep)[-4]
        if first_day != '01':
            print("Not first day data, skipped:", f)
            continue
        df = pd.read_csv(f)
        df_list.append(df)

    all_df = pd.concat(df_list)
    print("row count:", len(all_df))
    all_df.to_csv(fr'L:\SafeGraph_monthly_patterns_2018-2022\\{year}\US_monthly_home_panel_summary_{year}.csv', index=False)
all_df

Found files: 12
Year: 2018


C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning

row count: 2643603
Found files: 12
Year: 2019


C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


row count: 2643116
Found files: 12
Year: 2020
row count: 2643048
Found files: 12
Year: 2021


C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


row count: 2640227
Found files: 13
Year: 2022


C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\N\AppData\Local\Temp\ipykernel_26404\416220133.py:47: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


Not first day data, skipped: L:\SafeGraph_monthly_patterns_2018-2022\2022\11\22\SAFEGRAPH\MP\home_panel_summary.csv
row count: 2644588


,year,month,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime
0,2022,1,ak,US,20200002021,211,91
1,2022,1,ak,US,21300002001,142,95
2,2022,1,ak,US,20200015003,25,32
3,2022,1,ak,US,20900010002,74,70
4,2022,1,ak,US,20900019003,39,13
...,...,...,...,...,...,...,...
220203,2022,12,wy,US,560419753002,66,70
220204,2022,12,wy,US,560019636003,12,10
220205,2022,12,wy,US,560210015013,48,40
220206,2022,12,wy,US,560250011002,124,96


## Weekly

In [435]:
# weekly residing files
weekly_residing_all_df = pd.read_csv(r'J:\weekly_patterns_20211211\Panel_overview_data\WeeklyPattern\weekly_home_panel_summary.csv', dtype={'census_block_group':str})
# weekly_residing_df = weekly_residing_df.merge(sociala_variable_df, left_on='census_block_group', right_on='fips')
weekly_residing_all_df['census_block_group'] = weekly_residing_all_df['census_block_group'].str.zfill(12)
weekly_residing_all_df = weekly_residing_all_df.sort_values(['date_range_start', 'date_range_end'])
weekly_residing_all_df

,date_range_start,date_range_end,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime
54709667,2018-01-01T00:00:00+10:00,2018-01-08T00:00:00+10:00,gu,US,660109544001,5,5
54709668,2018-01-01T00:00:00+10:00,2018-01-08T00:00:00+10:00,gu,US,660109507022,5,4
54709669,2018-01-01T00:00:00+10:00,2018-01-08T00:00:00+10:00,gu,US,660109507023,14,12
54709670,2018-01-01T00:00:00+10:00,2018-01-08T00:00:00+10:00,gu,US,660109510002,4,4
54709671,2018-01-01T00:00:00+10:00,2018-01-08T00:00:00+10:00,gu,US,660109529003,3,3
...,...,...,...,...,...,...,...
16235,2021-12-27T00:00:00-10:00,2022-01-03T00:00:00-10:00,hi,US,150090311031,41,37
59971,2021-12-27T00:00:00-10:00,2022-01-03T00:00:00-10:00,ak,US,020160001001,34,22
65799,2021-12-27T00:00:00-11:00,2022-01-03T00:00:00-11:00,as,US,600109506002,5,6
65800,2021-12-27T00:00:00-11:00,2022-01-03T00:00:00-11:00,as,US,600109509002,1,0


In [1]:
YEAR = '2018'
if YEAR > "2019":
    acs_year = '2019'
else:
    acs_year = YEAR
acs_year

'2018'

In [475]:
social_variable_df = pd.read_csv(fr'F:\USC_OneDrive\OneDrive - University of South Carolina\Research\Safegraph_bias\cbg_acs_{acs_year}_county_tract.csv', dtype={'fips':str})
social_variable_df['fips'] = social_variable_df['fips'].astype(str).str.zfill(12)
social_variable_df.columns
print("US population, male, female:")
social_variable_df['totalpopulation'].sum(), social_variable_df['male'].sum(), social_variable_df['female'].sum(), len(social_variable_df)

US population, male, female:


(327939105, 161465237, 166551005, 220333)

In [476]:
weekly_residing_year_df = weekly_residing_all_df[weekly_residing_all_df["date_range_start"].str.startswith(YEAR)]
weekly_residing_merged_df = weekly_residing_year_df.merge(social_variable_df, left_on='census_block_group', right_on='fips')
weekly_residing_merged_df

,date_range_start,date_range_end,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,state_fips,county_fips,tract_fips,fips,state,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore
0,2021-01-04T00:00:00-04:00,2021-01-11T00:00:00-04:00,pr,US,720678204002,2,4,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0
1,2021-01-11T00:00:00-04:00,2021-01-18T00:00:00-04:00,pr,US,720678204002,2,4,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0
2,2021-01-18T00:00:00-04:00,2021-01-25T00:00:00-04:00,pr,US,720678204002,0,2,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0
3,2021-01-25T00:00:00-04:00,2021-02-01T00:00:00-04:00,pr,US,720678204002,0,1,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0
4,2021-02-01T00:00:00-04:00,2021-02-08T00:00:00-04:00,pr,US,720678204002,1,0,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11630225,2021-12-27T00:00:00-05:00,2022-01-03T00:00:00-05:00,mi,US,260139901000,1,0,26,26013,26013990100,260139901000,mi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11630226,2021-12-27T00:00:00-05:00,2022-01-03T00:00:00-05:00,ny,US,360470164000,1,0,36,36047,36047016400,360470164000,ny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11630227,2021-12-27T00:00:00-05:00,2022-01-03T00:00:00-05:00,wi,US,550039900000,1,0,55,55003,55003990000,550039900000,wi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11630228,2021-12-27T00:00:00-06:00,2022-01-03T00:00:00-06:00,la,US,220879900000,1,0,22,22087,22087990000,220879900000,la,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [477]:
weekly_residing_merged_df['date_range_start'] = weekly_residing_merged_df['date_range_start'].astype(str).str[:10]
weekly_residing_merged_df['state_fips'] = weekly_residing_merged_df['state_fips'].astype(str).str.zfill(2)
weekly_residing_merged_df['county_fips'] = weekly_residing_merged_df['county_fips'].astype(str).str.zfill(5)
weekly_residing_merged_df['tract_fips'] = weekly_residing_merged_df['tract_fips'].astype(str).str.zfill(11)
weekly_residing_merged_df['year'] =  weekly_residing_merged_df['date_range_start'].astype(str).str[:4]
weekly_residing_merged_df.rename(columns={'fips':'cbg_fips'}, inplace=True)
weekly_residing_merged_df

,date_range_start,date_range_end,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,state_fips,county_fips,tract_fips,cbg_fips,state,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore,year
0,2021-01-04,2021-01-11T00:00:00-04:00,pr,US,720678204002,2,4,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0,2021
1,2021-01-11,2021-01-18T00:00:00-04:00,pr,US,720678204002,2,4,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0,2021
2,2021-01-18,2021-01-25T00:00:00-04:00,pr,US,720678204002,0,2,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0,2021
3,2021-01-25,2021-02-01T00:00:00-04:00,pr,US,720678204002,0,1,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0,2021
4,2021-02-01,2021-02-08T00:00:00-04:00,pr,US,720678204002,1,0,72,72067,72067820400,720678204002,pr,684,325,359,23,11,20,0,58,72,55,67,127,103,124,22,186,76,0,0,0,371,51,684,572,127,167,92,49,6,0,0,285,69,87,36,21,31,6,5,0,0,4,26,0,0,0,0,0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11630225,2021-12-27,2022-01-03T00:00:00-05:00,mi,US,260139901000,1,0,26,26013,26013990100,260139901000,mi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021
11630226,2021-12-27,2022-01-03T00:00:00-05:00,ny,US,360470164000,1,0,36,36047,36047016400,360470164000,ny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021
11630227,2021-12-27,2022-01-03T00:00:00-05:00,wi,US,550039900000,1,0,55,55003,55003990000,550039900000,wi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021
11630228,2021-12-27,2022-01-03T00:00:00-06:00,la,US,220879900000,1,0,22,22087,22087990000,220879900000,la,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021


In [478]:
df = weekly_residing_merged_df[weekly_residing_merged_df["date_range_start"].str.startswith(YEAR)].query("region == 'sc'")
df

,date_range_start,date_range_end,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,state_fips,county_fips,tract_fips,cbg_fips,state,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore,year
150859,2021-01-04,2021-01-11T00:00:00-05:00,sc,US,450410003004,87,67,45,45041,45041000300,450410003004,sc,1866,829,1037,127,91,127,70,127,127,127,127,127,127,103,11,387,127,0,21,0,0,0,0,1156,127,301,127,127,46,4,3,580,67,11,51,34,21,16,47,29,10,70,108,60,39,11,4,2,2021
150860,2021-01-11,2021-01-18T00:00:00-05:00,sc,US,450410003004,83,67,45,45041,45041000300,450410003004,sc,1866,829,1037,127,91,127,70,127,127,127,127,127,127,103,11,387,127,0,21,0,0,0,0,1156,127,301,127,127,46,4,3,580,67,11,51,34,21,16,47,29,10,70,108,60,39,11,4,2,2021
150861,2021-01-18,2021-01-25T00:00:00-05:00,sc,US,450410003004,81,62,45,45041,45041000300,450410003004,sc,1866,829,1037,127,91,127,70,127,127,127,127,127,127,103,11,387,127,0,21,0,0,0,0,1156,127,301,127,127,46,4,3,580,67,11,51,34,21,16,47,29,10,70,108,60,39,11,4,2,2021
150862,2021-01-25,2021-02-01T00:00:00-05:00,sc,US,450410003004,83,65,45,45041,45041000300,450410003004,sc,1866,829,1037,127,91,127,70,127,127,127,127,127,127,103,11,387,127,0,21,0,0,0,0,1156,127,301,127,127,46,4,3,580,67,11,51,34,21,16,47,29,10,70,108,60,39,11,4,2,2021
150863,2021-02-01,2021-02-08T00:00:00-05:00,sc,US,450410003004,86,69,45,45041,45041000300,450410003004,sc,1866,829,1037,127,91,127,70,127,127,127,127,127,127,103,11,387,127,0,21,0,0,0,0,1156,127,301,127,127,46,4,3,580,67,11,51,34,21,16,47,29,10,70,108,60,39,11,4,2,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11630108,2021-10-18,2021-10-25T00:00:00-04:00,sc,US,450899708021,0,1,45,45089,45089970802,450899708021,sc,1266,1266,0,0,0,0,0,58,127,127,127,98,0,0,0,317,127,27,0,0,96,74,299,1208,127,473,127,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021
11630109,2021-10-25,2021-11-01T00:00:00-04:00,sc,US,450899708021,0,1,45,45089,45089970802,450899708021,sc,1266,1266,0,0,0,0,0,58,127,127,127,98,0,0,0,317,127,27,0,0,96,74,299,1208,127,473,127,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021
11630110,2021-11-01,2021-11-08T00:00:00-05:00,sc,US,450899708021,0,1,45,45089,45089970802,450899708021,sc,1266,1266,0,0,0,0,0,58,127,127,127,98,0,0,0,317,127,27,0,0,96,74,299,1208,127,473,127,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021
11630111,2021-11-08,2021-11-15T00:00:00-05:00,sc,US,450899708021,0,1,45,45089,45089970802,450899708021,sc,1266,1266,0,0,0,0,0,58,127,127,127,98,0,0,0,317,127,27,0,0,96,74,299,1208,127,473,127,37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021


In [479]:
import numpy as np
def get_sampling_rate(residing_df, aggre_columns):
    # residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]
    aggre_columns = ['date_range_start'] + aggre_columns
    
    sampling_rate_df = residing_df.groupby(aggre_columns).agg(
                                      
                                      start_date=('date_range_start', 'first'), 
                                      end_date=('date_range_start', 'last'), 
                                      # period_count=('CBG_count', 'count'), 
                                      CBG_count=('totalpopulation', 'count'), 
                                      fips=(aggre_columns[-1], 'first'), 
                                      device_count=('number_devices_residing', 'sum'), 
                                      population=('totalpopulation', 'sum'), 
                                      year=('year', 'first'),
                                      aggre_column=(aggre_columns[-1], 'first'), 
                                      # safegraph_ratio_max=('safegraph_ratio', max),
                                      # census_ratio=('census_ratio', 'mean'),
                                      # census_ratio_max=('census_ratio', max),
                                      # census_ratio_min=('census_ratio', min),
                                     )
    
    sampling_rate_df['sampling_rate'] = sampling_rate_df['device_count'] / sampling_rate_df['population']
    sampling_rate_df.replace([np.inf], np.nan, inplace=True)

    return sampling_rate_df.reset_index().iloc[:, 2:]


def save_df(df, save_name):
    
    print("Saved.")
    

def compute_sampling_rates(df, aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips']):
    df_list = []
    for c in aggre_columns:
        sampling_rate_df = get_sampling_rate(df, aggre_columns=[c])
        df_list.append(sampling_rate_df)
        
    df_all = pd.concat(df_list)
    
    return df_all

df_all = compute_sampling_rates(weekly_residing_merged_df, aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])
# df_all = compute_sampling_rates(df,                        aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])

df_all['fips_len'] = df_all['fips'].astype(str).str.len()
df_all['fips_level'] = df_all['fips_len'].astype(str)
df_all['fips_level'] = df_all['fips_level'].replace({'2':"state", '5':"county", '11': 'tract', '12':'cbg'})

df_all.to_csv(fr'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_{YEAR}.csv', index=False)
df_all

,start_date,end_date,CBG_count,fips,device_count,population,year,aggre_column,sampling_rate,fips_len,fips_level
0,2021-01-04,2021-01-04,3438,01,394059,4876250,2021,01,0.080812,2,state
1,2021-01-04,2021-01-04,528,02,22125,733893,2021,02,0.030147,2,state
2,2021-01-04,2021-01-04,4177,04,313834,7050299,2021,04,0.044514,2,state
3,2021-01-04,2021-01-04,2147,05,217957,2999370,2021,05,0.072668,2,state
4,2021-01-04,2021-01-04,23192,06,1311949,39272874,2021,06,0.033406,2,state
...,...,...,...,...,...,...,...,...,...,...,...
11410434,2021-12-27,2021-12-27,1,721537505032,2,981,2021,721537505032,0.002039,12,cbg
11410435,2021-12-27,2021-12-27,1,721537506011,2,883,2021,721537506011,0.002265,12,cbg
11410436,2021-12-27,2021-12-27,1,721537506012,7,2523,2021,721537506012,0.002774,12,cbg
11410437,2021-12-27,2021-12-27,1,721537506013,8,991,2021,721537506013,0.008073,12,cbg


In [480]:
df_all['sampling_rate'].mean(), df_all['sampling_rate'].max(), df_all['sampling_rate'].min()

(0.0539842884694264, 362.5, 0.0)

In [481]:
def time_aggregate(sampling_rate_df):
    
    rate_gb_df = sampling_rate_df.groupby(['year', 'fips']).agg(
                                      start_date=('start_date', 'first'), 
                                      end_date=('end_date', 'last'), 
                                      period_count=('start_date', 'count'), 
                                      CBG_count=('CBG_count', 'mean'), 
                                      device_count_mean=('device_count', 'mean'), 
                                      population=('population', 'mean'), 
                                      sampling_rate_mean=('sampling_rate', 'mean'),
                                      sampling_rate_max=('sampling_rate', max),
                                      sampling_rate_min=('sampling_rate', min),        
                                     )
    
    return rate_gb_df
    
rate_gb_df = time_aggregate(df_all.iloc[:]).reset_index()
# rate_gb_df.to_csv(r'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_yearly.csv', index=False)
rate_gb_df['fips_len'] = rate_gb_df['fips'].astype(str).str.len()
rate_gb_df['fips_level'] = rate_gb_df['fips_len'].astype(str)
rate_gb_df['fips_level'] = rate_gb_df['fips_level'].replace({'2':"state", '5':"county", '11': 'tract', '12':'cbg'})

rate_gb_df.to_csv(fr'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_{YEAR}_yearly.csv', index=False)

rate_gb_df

,year,fips,start_date,end_date,period_count,CBG_count,device_count_mean,population,sampling_rate_mean,sampling_rate_max,sampling_rate_min,fips_len,fips_level
0,2021,01,2021-01-04,2021-12-27,52,3503.615385,408712.230769,4.970024e+06,0.081901,0.103577,0.075744,2,state
1,2021,01001,2021-01-04,2021-12-27,52,32.615385,5263.519231,5.644500e+04,0.092875,0.115312,0.085645,5,county
2,2021,01001020100,2021-01-04,2021-12-27,52,2.038462,242.365385,2.031327e+03,0.118482,0.176618,0.100351,11,tract
3,2021,010010201001,2021-01-04,2021-12-27,52,1.019231,111.423077,7.440385e+02,0.148209,0.275342,0.093151,12,cbg
4,2021,010010201002,2021-01-04,2021-12-27,52,1.019231,130.942308,1.287288e+03,0.101300,0.123515,0.091053,12,cbg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
297408,2021,721537506012,2021-01-04,2021-12-27,49,1.020408,4.816327,2.574490e+03,0.001869,0.012287,0.000000,12,cbg
297409,2021,721537506013,2021-01-04,2021-12-27,50,1.020000,3.480000,1.010820e+03,0.003471,0.019173,0.000000,12,cbg
297410,2021,72153750602,2021-01-04,2021-12-27,45,1.488889,2.733333,1.831867e+03,0.001405,0.005393,0.000000,11,tract
297411,2021,721537506021,2021-01-04,2021-12-27,41,1.024390,1.951220,1.615463e+03,0.001206,0.005073,0.000000,12,cbg


In [482]:
rate_gb_df['sampling_rate_mean'].mean(), rate_gb_df['sampling_rate_mean'].max(), rate_gb_df['sampling_rate_mean'].min()

(0.05389302167150877, 151.57692307692307, 0.0)

In [483]:
stop

NameError: name 'stop' is not defined

## Monthly

In [3]:
# weekly residing files
residing_all_df = pd.read_csv(r'J:\monthly_patterns_20211211\Panel_overview_data\MonthlyPattern\monthly_home_panel_summary.csv', dtype={'census_block_group':str})
# weekly_residing_df = weekly_residing_df.merge(sociala_variable_df, left_on='census_block_group', right_on='fips')
residing_all_df['census_block_group'] = residing_all_df['census_block_group'].str.zfill(12)
residing_all_df['date_range_start'] = residing_all_df['year'].astype(str).str.zfill(4) + residing_all_df['month'].astype(str).str.zfill(2)
residing_all_df['date_range_end'] = residing_all_df['date_range_start'] 
residing_all_df = residing_all_df.sort_values(['date_range_start', 'date_range_end'])
residing_all_df['date_range_start'] = residing_all_df['year'].astype(str).str.zfill(4) + residing_all_df['month'].astype(str).str.zfill(2)
residing_all_df

,year,month,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,date_range_start,date_range_end
10569887,2018,1,ak,US,021880001002,9,5,201801,201801
10569888,2018,1,ak,US,021700006011,118,103,201801,201801
10569889,2018,1,ak,US,021220010001,120,93,201801,201801
10569890,2018,1,ak,US,021220006002,45,34,201801,201801
10569891,2018,1,ak,US,020200015003,42,38,201801,201801
...,...,...,...,...,...,...,...,...,...
220201,2022,1,wy,US,560119502001,86,60,202201,202201
220202,2022,1,wy,US,560199552003,62,52,202201,202201
220203,2022,1,wy,US,560159580003,83,54,202201,202201
220204,2022,1,wy,US,560250006002,90,67,202201,202201


In [4]:
YEAR = '2021'

if YEAR > "2019":
    acs_year = '2019'
else:
    acs_year = YEAR

In [5]:
residing_df = residing_all_df[residing_all_df["date_range_start"].astype(str).str.startswith(str(YEAR))] 
residing_df['date_range_start']

2640135    202101
2640136    202101
2640137    202101
2640138    202101
2640139    202101
            ...  
439951     202112
439952     202112
439953     202112
439954     202112
439955     202112
Name: date_range_start, Length: 2640227, dtype: object

In [530]:
social_variable_df = pd.read_csv(fr'F:\USC_OneDrive\OneDrive - University of South Carolina\Research\Safegraph_bias\cbg_acs_{acs_year}_county_tract.csv', dtype={'fips':str})
social_variable_df['fips'] = social_variable_df['fips'].astype(str).str.zfill(12)
social_variable_df.columns

print("acs_year, US population, male, female:")
acs_year, social_variable_df['totalpopulation'].sum(), social_variable_df['male'].sum(), social_variable_df['female'].sum(), len(social_variable_df)

acs_year, US population, male, female:


('2019', 327939105, 161465237, 166551005, 220333)

In [531]:
residing_merged_df = residing_df.merge(social_variable_df, left_on='census_block_group', right_on='fips')
residing_merged_df

,year,month,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,date_range_start,date_range_end,state_fips,county_fips,tract_fips,fips,state,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore
0,2021,1,ak,US,022400001002,28,22,202101,202101,2,2240,2240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
1,2021,2,ak,US,022400001002,15,14,202102,202102,2,2240,2240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
2,2021,3,ak,US,022400001002,21,21,202103,202103,2,2240,2240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
3,2021,4,ak,US,022400001002,26,18,202104,202104,2,2240,2240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
4,2021,5,ak,US,022400001002,31,22,202105,202105,2,2240,2240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2637927,2021,12,nc,US,370899801001,1,1,202112,202112,37,37089,37089980100,370899801001,nc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2637928,2021,12,nj,US,340297360010,1,0,202112,202112,34,34029,34029736001,340297360010,nj,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2637929,2021,12,ny,US,360850228000,1,0,202112,202112,36,36085,36085022800,360850228000,ny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2637930,2021,12,ny,US,360850244010,1,0,202112,202112,36,36085,36085024401,360850244010,ny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [532]:
residing_merged_df['date_range_start'] = residing_merged_df['date_range_start'].astype(str).str[:10]
residing_merged_df['state_fips'] = residing_merged_df['state_fips'].astype(str).str.zfill(2)
residing_merged_df['county_fips'] = residing_merged_df['county_fips'].astype(str).str.zfill(5)
residing_merged_df['tract_fips'] = residing_merged_df['tract_fips'].astype(str).str.zfill(11)
residing_merged_df['year'] =  residing_merged_df['date_range_start'].astype(str).str[:4]
residing_merged_df.rename(columns={'fips':'cbg_fips'}, inplace=True)
residing_merged_df

,year,month,region,iso_country_code,census_block_group,number_devices_residing,number_devices_primary_daytime,date_range_start,date_range_end,state_fips,county_fips,tract_fips,cbg_fips,state,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore
0,2021,1,ak,US,022400001002,28,22,202101,202101,02,02240,02240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
1,2021,2,ak,US,022400001002,15,14,202102,202102,02,02240,02240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
2,2021,3,ak,US,022400001002,21,21,202103,202103,02,02240,02240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
3,2021,4,ak,US,022400001002,26,18,202104,202104,02,02240,02240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
4,2021,5,ak,US,022400001002,31,22,202105,202105,02,02240,02240000100,022400001002,ak,1289,618,671,99,66,76,28,38,127,127,127,127,127,29,6,965,0,280,11,0,0,33,8,982,84,320,127,73,127,7,0,521,32,26,26,30,26,0,33,22,6,31,64,41,73,43,35,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2637927,2021,12,nc,US,370899801001,1,1,202112,202112,37,37089,37089980100,370899801001,nc,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2637928,2021,12,nj,US,340297360010,1,0,202112,202112,34,34029,34029736001,340297360010,nj,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2637929,2021,12,ny,US,360850228000,1,0,202112,202112,36,36085,36085022800,360850228000,ny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2637930,2021,12,ny,US,360850244010,1,0,202112,202112,36,36085,36085024401,360850244010,ny,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [533]:
import numpy as np
def get_sampling_rate(residing_df, aggre_columns):
    # residing_df['date_range_start'] = residing_df['date_range_start'].astype(str).str[:10]
    aggre_columns = ['date_range_start'] + aggre_columns
    
    sampling_rate_df = residing_df.groupby(aggre_columns).agg(
                                      
                                      start_date=('date_range_start', 'first'), 
                                      end_date=('date_range_start', 'last'), 
                                      # period_count=('CBG_count', 'count'), 
                                      CBG_count=('totalpopulation', 'count'), 
                                      fips=(aggre_columns[-1], 'first'), 
                                      device_count=('number_devices_residing', 'sum'), 
                                      population=('totalpopulation', 'sum'), 
                                      year=('year', 'first'),
                                      aggre_column=(aggre_columns[-1], 'first'), 
                                      # safegraph_ratio_max=('safegraph_ratio', max),
                                      # census_ratio=('census_ratio', 'mean'),
                                      # census_ratio_max=('census_ratio', max),
                                      # census_ratio_min=('census_ratio', min),
                                     )
    
    sampling_rate_df['sampling_rate'] = sampling_rate_df['device_count'] / sampling_rate_df['population']
    sampling_rate_df.replace([np.inf], np.nan, inplace=True)

    return sampling_rate_df.reset_index().iloc[:, 2:]


def save_df(df, save_name):
    
    print("Saved.")
    

def compute_sampling_rates(df, aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips']):
    df_list = []
    for c in aggre_columns:
        sampling_rate_df = get_sampling_rate(df, aggre_columns=[c])
        df_list.append(sampling_rate_df)
        
    df_all = pd.concat(df_list)
    
    return df_all

df_all = compute_sampling_rates(residing_merged_df, aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])
# df_all = compute_sampling_rates(df,                        aggre_columns = ['state_fips', 'county_fips', 'tract_fips', 'cbg_fips'])

df_all['fips_len'] = df_all['fips'].astype(str).str.len()
df_all['fips_level'] = df_all['fips_len'].astype(str)
df_all['fips_level'] = df_all['fips_level'].replace({'2':"state", '5':"county", '11': 'tract', '12':'cbg'})

df_all.to_csv(fr'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_{YEAR}_monthly.csv', index=False)
df_all

,start_date,end_date,CBG_count,fips,device_count,population,year,aggre_column,sampling_rate,fips_len,fips_level
0,202101,202101,3438,01,475154,4876250,2021,01,0.097443,2,state
1,202101,202101,533,02,31213,737054,2021,02,0.042348,2,state
2,202101,202101,4177,04,427128,7050299,2021,04,0.060583,2,state
3,202101,202101,2147,05,266447,2999370,2021,05,0.088834,2,state
4,202101,202101,23199,06,1821738,39277401,2021,06,0.046381,2,state
...,...,...,...,...,...,...,...,...,...,...,...
2637927,202112,202112,1,721537505032,1,981,2021,721537505032,0.001019,12,cbg
2637928,202112,202112,1,721537506011,1,883,2021,721537506011,0.001133,12,cbg
2637929,202112,202112,1,721537506012,1,2523,2021,721537506012,0.000396,12,cbg
2637930,202112,202112,1,721537506013,2,991,2021,721537506013,0.002018,12,cbg


In [534]:
df_all['sampling_rate'].mean(), df_all['sampling_rate'].max(), df_all['sampling_rate'].min() 

(0.07502739634419049, 639.5, 0.0)

In [ ]:
def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    # print("wt.sum:", wt.sum())
    return (val * wt).sum() / wt.sum() 

In [535]:
def time_aggregate(sampling_rate_df):
    
    rate_gb_df = sampling_rate_df.groupby(['year', 'fips']).agg(
                                      start_date=('start_date', 'first'), 
                                      end_date=('end_date', 'last'), 
                                      period_count=('start_date', 'count'), 
                                      CBG_count=('CBG_count', 'mean'), 
                                      device_count_mean=('device_count', 'mean'), 
                                      population=('population', 'mean'), 
                                      sampling_rate_mean=('sampling_rate', 'mean'),
                                      sampling_rate_max=('sampling_rate', max),
                                      sampling_rate_min=('sampling_rate', min),        
                                     )
    
    return rate_gb_df
    
rate_gb_df = time_aggregate(df_all.iloc[:]).reset_index()
# rate_gb_df.to_csv(r'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_yearly.csv', index=False)
rate_gb_df['fips_len'] = rate_gb_df['fips'].astype(str).str.len()
rate_gb_df['fips_level'] = rate_gb_df['fips_len'].astype(str)
rate_gb_df['fips_level'] = rate_gb_df['fips_level'].replace({'2':"state", '5':"county", '11': 'tract', '12':'cbg'})

rate_gb_df.to_csv(fr'P:\Shared drives\NJIT_T5\safegraph_bias\sampling_rates_{YEAR}_monthly_yearly.csv', index=False)

rate_gb_df

,year,fips,start_date,end_date,period_count,CBG_count,device_count_mean,population,sampling_rate_mean,sampling_rate_max,sampling_rate_min,fips_len,fips_level
0,2021,01,202101,202112,12,3437.750000,485747.166667,4876250.0,0.099615,0.116666,0.093924,2,state
1,2021,01001,202101,202112,12,32.000000,6097.916667,55380.0,0.110110,0.126761,0.104153,5,county
2,2021,01001020100,202101,202112,12,2.000000,349.916667,1993.0,0.175573,0.214250,0.147015,11,tract
3,2021,010010201001,202101,202112,12,1.000000,199.083333,730.0,0.272717,0.345205,0.197260,12,cbg
4,2021,010010201002,202101,202112,12,1.000000,150.833333,1263.0,0.119425,0.138559,0.111639,12,cbg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
297357,2021,721537506012,202101,202112,12,1.000000,8.916667,2523.0,0.003534,0.013872,0.000396,12,cbg
297358,2021,721537506013,202101,202112,12,1.000000,6.416667,991.0,0.006475,0.023209,0.001009,12,cbg
297359,2021,72153750602,202101,202112,12,1.833333,4.583333,2117.0,0.002091,0.006292,0.000000,11,tract
297360,2021,721537506021,202101,202112,12,1.000000,2.666667,1577.0,0.001691,0.005073,0.000000,12,cbg


In [536]:
rate_gb_df['sampling_rate_mean'].mean(), rate_gb_df['sampling_rate_mean'].max(), rate_gb_df['sampling_rate_mean'].min()

(0.07498375601505249, 270.125, 0.0)

In [537]:
stop

NameError: name 'stop' is not defined

In [ ]:
df = pd.DataFrame(columns=['C1', 'C2', "C3"])
df["C1"] = [2, 2, 2, 1, 1, 1]
df["C2"] = ["a"] * 2 + ["b"] * 2 + ["c"] * 2
df["C3"] = range(6)

df = df.groupby(['C1', 'C2']).agg(C4=('C3', 'mean'),)

df.loc[:, :]

In [ ]:
df.loc[:, ['a', 'c'], :]

In [ ]:
bias_gb_df.index.get_level_values(1)
bias_gb_df.loc[:, ['male', 'female'], :]

In [ ]:
# SC_residing_df.sort_values('date_range_start')

In [ ]:
# weekly residing files
weekly_residing_df = pd.read_csv(r'J:\weekly_patterns_20211211\Panel_overview_data\WeeklyPattern\weekly_home_panel_summary.csv')
weekly_residing_df

In [ ]:
ACS_csv = r'H:\covid_mobility_results\new_census_data\ACS_2019_5YR_BG\ACS_race_cbsa_income_2019_county_decile.csv'

ACS_df = pd.read_csv(ACS_csv)


sociala_variable_df = pd.read_csv(r'F:\USC_OneDrive\OneDrive - University of South Carolina\Research\Mobility_covid\social_variables.csv')


sociala_variable_df
sociala_variable_df['GEOID'] = sociala_variable_df['GEOID'].str[-12:]

In [ ]:
# sociala_variable_df

In [ ]:
def text2GEOID(text):
    # for state, county, tract, blockgroup level only
    county_fips = ''
    tract_fips = ''
    blockgroup_fips = ''    
    state_fips = ''
    try:
        state_fips = text.split('state:')[1][:2]
        county_fips = text.split('county:')[1][:3]
        tract_fips = text.split('tract:')[1][:6]
        blockgroup_fips = text.split('block group:')[1][:1]
        
    except Exception as e:
        print("Error in text2GEOID()", e)
        pass
    
    return state_fips + county_fips + tract_fips + blockgroup_fips
                            
# make a test
text = r'Block Group 1, Census Tract 113, Coffee County, Alabama: Summary level: 150, state:01> county:031> tract:011300> block group:1'

len(text2GEOID(text))
                            

In [ ]:
# B01001_026E: SEX BY AGE: Female: Total population -- (Estimate)

# B01001_001E: SEX BY AGE: Total: Total population -- (Estimate)

def get_census(state_fips=['45'], county_fips=[], blockgroup_fips=[], fields=['B01001_001E', 'B01001_026E']):  
    # unfinished
    
    df_list = []
    
    for state in state_fips:
        try:
            cookbg = censusdata.download('acs5', 2019,
                                         censusdata.censusgeo([('state', state), ('county', '*'), ('block group', '*')]),
                                         ['B01001_001E', 'B01001_026E'])

            index = [text2GEOID(str(t)) for t in cookbg.index]
            cookbg.index = index
            cookbg.index = cookbg.index.set_names(['FIPS'])
            cookbg = cookbg.reset_index()
            df_list.append(cookbg)
        except Exception as e:
            print("Error in get Census data in :", state, e)
            continue
        
    return pd.concat(df_list)

# make a test

cookbg = get_census(state_fips=['45'], county_fips=[], blockgroup_fips=[])
len(cookbg)

In [ ]:
cookbg['Female_ratio']#mean()

In [ ]:
# state_fips = list(states.mapping('fips', 'abbr').keys())[:-1] # the last one is none, drop it using [:-1].

# get data of SC
state_fips = ['45'] # the last one is none, drop it using [:-1]
cookbg = get_census(state_fips=state_fips, county_fips=[], blockgroup_fips=[])

cookbg['ratio_female'] = cookbg['B01001_026E'] / cookbg['B01001_001E']


len(cookbg), cookbg['ratio_female'].mean()

cookbg['ratio_female']

In [ ]:
cookbg = cookbg.merge(sociala_variable_df, left_on=['FIPS'], right_on=['GEOID'])

In [ ]:
SC_residing_df = weekly_residing_df[weekly_residing_df["date_range_start"].str.startswith('2019')].query("region == 'sc'")
SC_residing_df

In [ ]:
SC_residing_df = SC_residing_df.merge(cookbg, left_on='census_block_group', right_on='FIPS')
SC_residing_df = SC_residing_df.query('B01001_001E > 1').dropna()
SC_residing_df

In [ ]:
columns = [ 'ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_high_school', 'ratio_senior']
columns = ['ratio_female', 'ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_high_school', 'ratio_senior']


def get_bias(df):
    
    # columns = ['ratio_female', 'ratio_white', 'ratio_black', 'ratio_hispanic', 'ratio_high_school', 'ratio_senior']
    state_total_device = df['number_devices_residing'].sum()
    state_total_pop = df['popu_cbg'].sum()
    
    result_df = pd.DataFrame(columns = [ 'variable', 'start_date', 'census_ratio', 'safegraph_ratio'],
                   index = columns)
    
    for column in columns:
        start_date = df.iloc[0]['date_range_start'][:10] 
        cbg_column_devices = df['number_devices_residing'] * df[column]
        cbg_device_ratio = cbg_column_devices.sum() / state_total_device 

        cbg_column_pop = df[column] * df['popu_cbg']
        
        census_ratio = cbg_column_pop.sum() / state_total_pop
        safegraph_ratio = cbg_column_devices.sum() / state_total_device
        
        result_df.loc[column]['variable'] = column
        result_df.loc[column]['start_date'] = start_date
        result_df.loc[column]['census_ratio'] = census_ratio
        result_df.loc[column]['safegraph_ratio'] = safegraph_ratio
        
        
        # print(f"{safegraph_ratio:0.3f}, {census_ratio:.3f}")
    # print(result_df)
    return result_df
bias_df = SC_residing_df.groupby('date_range_start').apply(get_bias)
bias_df.reset_index(drop=True, inplace=True)

# bias_df = bias_df[columns]


bias_gb_df = bias_df.groupby(['variable']).agg(census_ratio_mean=('safegraph_ratio', 'mean'), 
                                  census_ratio_min=('safegraph_ratio', min), 
                                  census_ratio_max=('safegraph_ratio', max),
                                  census_ratio=('census_ratio', max),
                                 )

bias_gb_df = bias_gb_df.loc[columns]

bias_gb_df['census_ratio_mean_to_census'] = bias_gb_df['census_ratio_mean'] / bias_gb_df['census_ratio']
bias_gb_df['census_ratio_min_to_census'] = bias_gb_df['census_ratio_min'] / bias_gb_df['census_ratio']
bias_gb_df['census_ratio_max_to_census'] = bias_gb_df['census_ratio_max'] / bias_gb_df['census_ratio']

bias_gb_df.round(2)

In [ ]:
# 

In [ ]:
bias_df.reset_index(drop=True, inplace=True)

In [ ]:
SC_residing_df.columns

In [ ]:
cookbg

In [ ]:


# cookbg

In [ ]:
CENSUS_KEY = 'b12026d61228a4b0d441ae7aa93f1ea222877503'

# two packages can be use: Census (need a Census key), Censusdata (no key needed)


from census import Census
from us import states

c = Census(CENSUS_KEY)
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

In [ ]:
Census.ALL

In [ ]:
# c.sf1.state_county_blockgroup('NAME', states.AK.fips, '170', Census.ALL)
